# English-to-Spanish translation with KerasHub

**Author:** [Abheesht Sharma](https://github.com/abheesht17/)<br>
**Date created:** 2022/05/26<br>
**Last modified:** 2024/04/30<br>
**Description:** Use KerasHub to train a sequence-to-sequence Transformer model on the machine translation task.

## Introduction

KerasHub provides building blocks for NLP (model layers, tokenizers, metrics, etc.) and
makes it convenient to construct NLP pipelines.

In this example, we'll use KerasHub layers to build an encoder-decoder Transformer
model, and train it on the English-to-Spanish machine translation task.

This example is based on the
[English-to-Spanish NMT
example](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/)
by [fchollet](https://twitter.com/fchollet). The original example is more low-level
and implements layers from scratch, whereas this example uses KerasHub to show
some more advanced approaches, such as subword tokenization and using metrics
to compute the quality of generated translations.

You'll learn how to:

- Tokenize text using `keras_hub.tokenizers.WordPieceTokenizer`.
- Implement a sequence-to-sequence Transformer model using KerasHub's
`keras_hub.layers.TransformerEncoder`, `keras_hub.layers.TransformerDecoder` and
`keras_hub.layers.TokenAndPositionEmbedding` layers, and train it.
- Use `keras_hub.samplers` to generate translations of unseen input sentences
 using the top-p decoding strategy!

Don't worry if you aren't familiar with KerasHub. This tutorial will start with
the basics. Let's dive right in!

## Setup

Before we start implementing the pipeline, let's import all the libraries we need.

In [1]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-hub
!pip install -q --upgrade keras  # Upgrade to Keras 3.

In [2]:
import keras_hub
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)

Let's also define our parameters/hyperparameters.

In [3]:
BATCH_SIZE = 64
EPOCHS = 100  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 20
ENG_VOCAB_SIZE = 20000
SPA_VOCAB_SIZE = 20000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

## Downloading the data

We'll be working with an English-to-Spanish translation dataset
provided by [Anki](https://www.manythings.org/anki/). Let's download it:

In [4]:
from google.colab import drive

drive.mount('/content/drive')
text_file = '/content/drive/MyDrive/Colab Notebooks/corpus/spa.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Parsing the data

Each line contains an English sentence and its corresponding Spanish sentence.
The English sentence is the *source sequence* and Spanish one is the *target sequence*.
Before adding the text to a list, we convert it to lowercase.

In [5]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = eng.lower()
    spa = spa.lower()
    text_pairs.append((eng, spa))

Here's what our sentence pairs look like:

In [6]:
for _ in range(5):
    print(random.choice(text_pairs))

('most students like sports.', 'a la mayoría de los estudiantes les gustan los deportes.')
('he always borrows money from me.', 'él siempre me pide dinero prestado.')
("i'd like to send my dress to a dry cleaner.", 'me gustaría enviar mi vestido a una tintorería.')
('you lack imagination.', 'te falta imaginación.')
('i saw tom in the mirror so i turned around and said hi.', 'vi a tom reflejado en el espejo, así que le dije hola.')


Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [7]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")


141543 total pairs
99081 training pairs
21231 validation pairs
21231 test pairs


## Tokenizing the data

We'll define two tokenizers - one for the source language (English), and the other
for the target language (Spanish). We'll be using
`keras_hub.tokenizers.WordPieceTokenizer` to tokenize the text.
`keras_hub.tokenizers.WordPieceTokenizer` takes a WordPiece vocabulary
and has functions for tokenizing the text, and detokenizing sequences of tokens.

Before we define the two tokenizers, we first need to train them on the dataset
we have. The WordPiece tokenization algorithm is a subword tokenization algorithm;
training it on a corpus gives us a vocabulary of subwords. A subword tokenizer
is a compromise between word tokenizers (word tokenizers need very large
vocabularies for good coverage of input words), and character tokenizers
(characters don't really encode meaning like words do). Luckily, KerasHub
makes it very simple to train WordPiece on a corpus with the
`keras_hub.tokenizers.compute_word_piece_vocabulary` utility.

In [8]:

def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_hub.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab


Every vocabulary has a few special, reserved tokens. We have four such tokens:

- `"[PAD]"` - Padding token. Padding tokens are appended to the input sequence
length when the input sequence length is shorter than the maximum sequence length.
- `"[UNK]"` - Unknown token.
- `"[START]"` - Token that marks the start of the input sequence.
- `"[END]"` - Token that marks the end of the input sequence.

In [9]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

spa_samples = [text_pair[1] for text_pair in train_pairs]
spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)

Let's see some tokens!

In [10]:
print("English Tokens: ", eng_vocab[100:110])
print("Spanish Tokens: ", spa_vocab[100:110])

English Tokens:  ['go', 'him', 'll', 'they', 'how', 'has', 'did', 'her', 'time', 'as']
Spanish Tokens:  ['ella', 'las', 'más', 'al', 'yo', 'eso', 'tu', 'estoy', 'muy', 'esta']


Now, let's define the tokenizers. We will configure the tokenizers with the
the vocabularies trained above.

In [11]:
eng_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
spa_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=spa_vocab, lowercase=False
)

Let's try and tokenize a sample from our dataset! To verify whether the text has
been tokenized correctly, we can also detokenize the list of tokens back to the
original text.

In [12]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

print()

spa_input_ex = text_pairs[0][1]
spa_tokens_ex = spa_tokenizer.tokenize(spa_input_ex)
print("Spanish sentence: ", spa_input_ex)
print("Tokens: ", spa_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    spa_tokenizer.detokenize(spa_tokens_ex),
)

English sentence:  i pretended that i didn't know tom.
Tokens:  tf.Tensor([  34 1463   69   34  112    8   45   94   65   11], shape=(10,), dtype=int32)
Recovered text after detokenizing:  i pretended that i didn ' t know tom .

Spanish sentence:  hice como si no conociera a tom.
Tokens:  tf.Tensor([ 409  122  115   76   93  302 1632   28   77   13], shape=(10,), dtype=int32)
Recovered text after detokenizing:  hice como si no conociera a tom .


## Format datasets

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the tokenized source sentence and `decoder_inputs` is the target
sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target
sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

We will add special tokens, `"[START]"` and `"[END]"`, to the input Spanish
sentence after tokenizing the text. We will also pad the input to a fixed length.
This can be easily done using `keras_hub.layers.StartEndPacker`.

In [13]:

def preprocess_batch(eng, spa):
    batch_size = ops.shape(spa)[0]

    eng = eng_tokenizer(eng)
    spa = spa_tokenizer(spa)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `spa` and pad it as well.
    spa_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=spa_tokenizer.token_to_id("[START]"),
        end_value=spa_tokenizer.token_to_id("[END]"),
        pad_value=spa_tokenizer.token_to_id("[PAD]"),
    )
    spa = spa_start_end_packer(spa)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 40 steps long):

In [14]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


## Building the model

Now, let's move on to the exciting part - defining our model!
We first need an embedding layer, i.e., a vector for every token in our input sequence.
This embedding layer can be initialised randomly. We also need a positional
embedding layer which encodes the word order in the sequence. The convention is
to add these two embeddings. KerasHub has a `keras_hub.layers.TokenAndPositionEmbedding `
layer which does all of the above steps for us.

Our sequence-to-sequence Transformer consists of a `keras_hub.layers.TransformerEncoder`
layer and a `keras_hub.layers.TransformerDecoder` layer chained together.

The source sequence will be passed to `keras_hub.layers.TransformerEncoder`, which
will produce a new representation of it. This new representation will then be passed
to the `keras_hub.layers.TransformerDecoder`, together with the target sequence
so far (target words 0 to N). The `keras_hub.layers.TransformerDecoder` will
then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking.
The `keras_hub.layers.TransformerDecoder` sees the entire sequence at once, and
thus we must make sure that it only uses information from target tokens 0 to N
when predicting token N+1 (otherwise, it could use information from the future,
which would result in a model that cannot be used at inference time). Causal masking
is enabled by default in `keras_hub.layers.TransformerDecoder`.

We also need to mask the padding tokens (`"[PAD]"`). For this, we can set the
`mask_zero` argument of the `keras_hub.layers.TokenAndPositionEmbedding` layer
to True. This will then be propagated to all subsequent layers.

In [15]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_hub.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=SPA_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_hub.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(SPA_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

## Training our model

We'll use accuracy as a quick way to monitor training progress on the validation data.
Note that machine translation typically uses BLEU scores as well as other metrics,
rather than accuracy. However, in order to use metrics like ROUGE, BLEU, etc. we
will have decode the probabilities and generate the text. Text generation is
computationally expensive, and performing this during training is not recommended.

Here we only train for 1 epoch, but to get the model to actually converge
you should train for at least 10 epochs.

In [16]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

transformer.summary()
transformer.compile(
    "rmsprop",
   loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["accuracy"]
)

# csv logger callback
csv_logger = CSVLogger("/content/drive/MyDrive/Colab Notebooks/transformer.training.log")

# model checkpoint callback
model_checkpoint = ModelCheckpoint(
    filepath="/content/drive/MyDrive/Colab Notebooks/transformer.checkpoint.keras",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)
transformer.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    verbose=2,
    callbacks=[csv_logger, model_checkpoint],
)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_and_position_embed… │ (None, None, 256)      │      5,125,120 │ encoder_inputs[0][0]   │
│ (TokenAndPositionEmbeddi… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      1,315,072 │ token_and_position_em… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_1 (Functional) │ (None, None, 20000)    │     11,843,872 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 18,284,064 (69.75 MB)

 Trainable params: 18,284,064 (69.75 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1549/1549 - 85s - 55ms/step - accuracy: 0.1851 - loss: 4.0011 - val_accuracy: 0.2449 - val_loss: 2.9349
Epoch 2/100
1549/1549 - 21s - 13ms/step - accuracy: 0.2545 - loss: 2.7890 - val_accuracy: 0.2737 - val_loss: 2.4799
Epoch 3/100
1549/1549 - 21s - 13ms/step - accuracy: 0.2765 - loss: 2.4401 - val_accuracy: 0.2872 - val_loss: 2.2819
Epoch 4/100
1549/1549 - 21s - 13ms/step - accuracy: 0.2894 - loss: 2.2596 - val_accuracy: 0.2956 - val_loss: 2.2095
Epoch 5/100
1549/1549 - 20s - 13ms/step - accuracy: 0.2985 - loss: 2.1386 - val_accuracy: 0.2989 - val_loss: 2.1730
Epoch 6/100
1549/1549 - 20s - 13ms/step - accuracy: 0.3049 - loss: 2.0588 - val_accuracy: 0.3022 - val_loss: 2.1566
Epoch 7/100
1549/1549 - 20s - 13ms/step - accuracy: 0.3098 - loss: 1.9968 - val_accuracy: 0.3022 - val_loss: 2.1590
Epoch 8/100
1549/1549 - 20s - 13ms/step - accuracy: 0.3142 - loss: 1.9410 - val_accuracy: 0.3043 - val_loss: 2.1568
Epoch 9/100
1549/1549 - 20s - 13ms/step - accuracy: 0.3176 - loss: 1.896